In [1]:
'''
! pip install langchain
! pip install openai
'''

'\n! pip install langchain\n! pip install openai\n'

In [2]:
import config
import os

os.environ['OPENAI_API_KEY'] = config.API_KEY

In [3]:
# from langchain.llms import OpenAI

# llm_opemAI = OpenAI(temperature=0.9)
# print(llm_opemAI.predict("What is the Capital of india?"))
# print(llm_opemAI("What would be a good company name for a company that makes colorful socks?"))

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

In [5]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Happy Socksy.


## Prompt 

In [6]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [7]:
prompt = PromptTemplate(
    input_variables=["Programming_language"],
    template="What are the top 5 Programming language youtube channel to learn {Programming_language}",
)

In [8]:
print(prompt.format(Programming_language="python"))
print(llm(prompt.format(Programming_language="python")))

What are the top 5 Programming language youtube channel to learn python


1. Sentdex 
2. freeCodeCamp.org 
3. Corey Schafer 
4. Telusko 
5. Traversy Media


In [10]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """


In [ ]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="")





# prompt_template = "What is a good name for a company that makes {product}?"

# llm = OpenAI(temperature=0)
# llm_chain = LLMChain(
#     llm=llm,
#     prompt=PromptTemplate.from_template(prompt_template)
# )
# llm_chain("colorful socks")

## Chaining

In [ ]:
from langchain.chains import LLMChain

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run('Java'))